# 2.4
Separate notebook as `ch2.ipynb` got large and unwieldly/slow.

## Complex-Number System

The complex number $z=x+iy$ (where $i^2=-1$) can be thought of as the point in the plane whose *real* coordinate is $x$ and whose *imaginary* coordinate is $y$.

Can have 2 representations:
- Reactangular form
    - Real part and imaginary part
- Polar form
    - Magnitude and angle
    
![](images/complex-numbers.png)

#### Addition 
Of two complex numbers $z_1,z_2$ reduces to addition of coordinates - using rectangular form:

$RealPart(z_1+z_2)=RealPart(z_1)+RealPart(z_2)$

$ImaginaryPart(z_1+z_2)=ImaginaryPart(z_1)+ImaginaryPart(z_2)$

#### Product 
Of two complex numbers $z_1,z_2$ is the vector obtained by stretching one complex number by the length of the other and then rotating it throught the angle of the other - using polar form:

$Magnitude(z_1\cdot z_2) = Magnitude(z_1)\cdot Magnitude(z_2)$

$Angle(z_1\cdot z_2) = Angle(z_1)+Magnitude(z_2)$

In [4]:
;; ASSUME put and get procedures are implemented in the language
(define (install-rectangular-package)  
  ;; internal procedures
  (define (real-part z) (car z))
  (define (imag-part z) (cdr z))
  (define (make-from-real-imag x y)
    (cons x y))
  (define (magnitude z)
    (sqrt (+ (square (real-part z))
             (square (imag-part z)))))
  (define (angle z)
    (atan (imag-part z) (real-part z)))
  (define (make-from-mag-ang r a)
    (cons (* r (cos a)) (* r (sin a))))
  ;; interface to the rest of the system
  (define (tag x)
    (attach-tag 'rectangular x))
  (put 'real-part '(rectangular) real-part)
  (put 'imag-part '(rectangular) imag-part)
  (put 'magnitude '(rectangular) magnitude)
  (put 'angle '(rectangular) angle)
  (put 'make-from-real-imag 'rectangular
       (lambda (x y)
         (tag (make-from-real-imag x y))))
  (put 'make-from-mag-ang 'rectangular
       (lambda (r a)
         (tag (make-from-mag-ang r a))))
  'done)

(define (install-polar-package)
  ;; internal procedures
  (define (magnitude z) (car z))
  (define (angle z) (cdr z))
  (define (make-from-mag-ang r a) (cons r a))
  (define (real-part z)
    (* (magnitude z) (cos (angle z))))
  (define (imag-part z)
    (* (magnitude z) (sin (angle z))))
  (define (make-from-real-imag x y)
    (cons (sqrt (+ (square x) (square y)))
          (atan y x)))
  ;; interface to the rest of the system
  (define (tag x) (attach-tag 'polar x))
  (put 'real-part '(polar) real-part)
  (put 'imag-part '(polar) imag-part)
  (put 'magnitude '(polar) magnitude)
  (put 'angle '(polar) angle)
  (put 'make-from-real-imag 'polar
       (lambda (x y)
         (tag (make-from-real-imag x y))))
  (put 'make-from-mag-ang 'polar
       (lambda (r a)
         (tag (make-from-mag-ang r a))))
  'done)

(define (apply-generic op . args)
  (let ((type-tags (map type-tag args)))
    (let ((proc (get op type-tags)))
      (if proc
          (apply proc (map contents args))
          (error
           "No method for these types:
           APPLY-GENERIC"
           (list op type-tags))))))

;; generic selectors
(define (real-part z)
  (apply-generic 'real-part z))
(define (imag-part z)
  (apply-generic 'imag-part z))
(define (magnitude z)
  (apply-generic 'magnitude z))
(define (angle z)
  (apply-generic 'angle z))

;; generic constructors
(define (make-from-real-imag x y)
  ((get 'make-from-real-imag
        'rectangular)
   x y))
(define (make-from-mag-ang r a)
  ((get 'make-from-mag-ang
        'polar)
   r a))

## 2.73

In [6]:
;; original deriv implementation from 2.3.2 for reference
;; other procedures used defined in ch2.ipynb
(define (deriv exp var)
  (cond ((number? exp) 0)
        ((variable? exp)
         (if (same-variable? exp var) 1 0))
        ((sum? exp)
         (make-sum (deriv (addend exp) var)
                   (deriv (augend exp) var)))
        ((product? exp)
         (make-sum
          (make-product
           (multiplier exp)
           (deriv (multiplicand exp) var))
          (make-product
           (deriv (multiplier exp) var)
           (multiplicand exp))))
        ;; more derivation rules here
        (else (error "unknown expression type:
                     DERV" exp))))

;; data-directed style deriv
(define (deriv exp var)
  (cond ((number? exp) 0)
        ((variable? exp)
         (if (same-variable? exp var)
             1
             0))
        (else ((get 'deriv (operator exp))
               (operands exp)
               var))))

(define (operator exp) (car exp))
(define (operands exp) (cdr exp))

The logic for implementing different derivation rules based on the type of expression passed to `deriv` has been move out of the body of `deriv`- with a separate procedure defined for each type of expression. These are installed in the dispatch table with the *operator* of the expression used as their type-tag. `deriv` then retrieves the appropriate procedure based on the operator of `exp` and applies it to the operands of `exp` and the given variable.

`number?` and `variable?` can't be assimilated into the data-directed dispatch as they must be able to act on single values which do not contain operators (only a number or a symbol). This means that a type-tag can't be assigned to them to retrieve the procedure from the dispatch table using `get`.

In [8]:
(define (deriv-sum exp var)
  (make-sum (deriv (addend exp) var)
            (deriv (augend exp) var)))

(define (deriv-product exp var)
  (make-sum
   (make-product
    (multiplier exp)
    (deriv (multiplicand exp) var))
   (make-product
    (deriv (multiplier exp) var)
    (multiplicand exp))))

(define (install-deriv-procs)
  (put 'deriv '+ deriv-sum)
  (put 'deriv '* deriv-product)
  'done)

In [ ]:
(define (deriv-exponentiation exp var)
  (make-product
   (make-product (exponent exp)
                 (make-exponentiation (base exp)
                                      (- (exponent exp) 1)))
   (deriv (base exp) var)))

(define (install-deriv-exponentiation)
  (put 'deriv '** deriv-exponentiation)
  'done)

Only changes are to swap the order of the procedure and type-tag/operator in the installation procedures:
```scheme
(put '+ 'deriv deriv-sum)
(put '* 'deriv deriv-product)
(put '** 'deriv deriv-exponentiation)
```